In [1]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
from tqdm import tqdm
import time
import random
import os
from IPython.display import display, HTML
import csv
from tqdm.notebook import tqdm
import re

In [2]:
def scrape_data(url, year, team_name):
    
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"An error occurred while fetching the URL: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')

    # Team General
    try:
        details = {
            "Record": soup.find('p').text,
            "Coach": soup.find_all('p')[1].text,
            "Points For": soup.find_all('p')[2].text,
            "Points Against": soup.find_all('p')[3].text,
            "Expected W-L": soup.find_all('p')[4].text,
            "SRS/SOS": soup.find_all('p')[5].text,
            "Playoffs": soup.find_all('p')[6].text,
            "Offensive Coordinator": soup.find_all('p')[7].text,
            "Defensive Coordinator": soup.find_all('p')[8].text,
            "Other Notable Asst.": soup.find_all('p')[9].text,
            "Stadium": soup.find_all('p')[10].text,
            "Owner/President/GM": soup.find_all('p')[11].text,
            "Offensive Scheme": soup.find_all('p')[12].text,
            "Defensive Alignment": soup.find_all('p')[13].text,
            "Preseason Odds": soup.find_all('p')[14].text,
            "Training Camp": soup.find_all('p')[15].text
        }
        with open(f'./data-teams/{team_name}_{year}_team_general.csv', 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                writer.writerow(details.keys())
            writer.writerow(details.values())
    except Exception as e:
        print(f"An error occurred while writing the team general CSV file: {e}")

    # Team Stats and Rankings
    try:
        table = soup.find('table', {'id': 'team_stats'})
        headers = [th.text for th in table.find('thead').find_all('tr')[1].find_all('th')]
        rows = []
        for tr in table.find('tbody').find_all('tr'):
            row = [td.text for td in tr.find_all('td')]
            rows.append(row)
        with open(f'./data-teams/{team_name}_{year}_team_stats.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                writer.writerow(headers[1:])
            writer.writerows(rows)
    except Exception as e:
        print(f"An error occurred while writing the team stats CSV file: {e}")

    # Teams Schedule and Game Results
    try:
        table = soup.find_all('table')[1]
        rows = table.find_all('tr')
        data = []
        for row in rows:
            cols = row.find_all('td')
            cols = [col.text for col in cols]
            data.append(cols)
        with open(f'./data-teams/{team_name}_{year}_team_schedule.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                writer.writerow(headers)
            writer.writerows(data)
    except Exception as e:
        print(f"An error occurred while writing the team schedule CSV file: {e}")

    # Team Conversions
    try:
        table = soup.find('table', {'id': 'team_conversions'})
        headers = [th.text for th in table.find('thead').find_all('th')]
        rows = []
        for tr in table.find('tbody').find_all('tr'):
            row = [td.text for td in tr.find_all('td')]
            rows.append(row)
        with open(f'./data-teams/{team_name}_{year}_team_conversions.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                writer.writerow(headers[1:])
            writer.writerows(rows)
    except Exception as e:
        print(f"An error occurred while writing the team conversions CSV file: {e}")

    # Passing Stats
    try:
        passing_div = soup.find('div', id='all_passing')
        passing_table = BeautifulSoup(str(passing_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
        with open(f'./data-teams/{team_name}_{year}_passing_data.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                main_header_row = passing_table.find('tr')
                main_headers = [col.text for col in main_header_row.find_all('th') if col.text.strip()]
                writer.writerow(main_headers)
            data_rows = passing_table.find('tbody').find_all('tr')
            for row in data_rows:
                data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(data)
    except Exception as e:
        print(f"An error occurred while writing the passing data CSV file: {e}")

    # Rushing and Receiving
    try:
        rushing_receiving_div = soup.find('div', id='all_rushing_and_receiving')
        rushing_receiving_table = BeautifulSoup(str(rushing_receiving_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
        with open(f'./data-teams/{team_name}_{year}_rushing_receiving_data.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                main_header_row = rushing_receiving_table.find_all('tr')[1]  # The second row contains the main headers
                main_headers = [col.text for col in main_header_row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(main_headers)
            data_rows = rushing_receiving_table.find('tbody').find_all('tr')
            for row in data_rows:
                data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(data)
    except Exception as e:
        print(f"An error occurred while writing the rushing receiving data CSV file: {e}")

    # Kicking
    try:
        kicking_div = soup.find('div', id='all_kicking')
        kicking_table = BeautifulSoup(str(kicking_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
        with open(f'./data-teams/{team_name}_{year}_kicking_data.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                main_header_row = kicking_table.find('tr')
                main_headers = [col.text for col in main_header_row.find_all('th') if col.text.strip()]
                writer.writerow(main_headers)
            data_rows = kicking_table.find('tbody').find_all('tr')
            for row in data_rows:
                data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(data)
    except Exception as e:
        print(f"An error occurred while writing the kicking data CSV file: {e}")

    # Punting
    try:
        punting_div = soup.find('div', id='all_punting')
        punting_table = BeautifulSoup(str(punting_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
        with open(f'./data-teams/{team_name}_{year}_punting_data.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                main_header_row = punting_table.find('tr')
                main_headers = [col.text for col in main_header_row.find_all('th') if col.text.strip()]
                writer.writerow(main_headers)
            data_rows = punting_table.find('tbody').find_all('tr')
            for row in data_rows:
                data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(data)
    except Exception as e:
        print(f"An error occurred while writing the punting data CSV file: {e}")

    # Defense and Fumbles
    try:
        defense_div = soup.find('div', id='all_defense')
        defense_table = BeautifulSoup(str(defense_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
        with open(f'./data-teams/{team_name}_{year}_defense_fumbles_data.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                main_header_row = defense_table.find_all('tr')[2]  # The third row contains the main headers
                main_headers = [col.text for col in main_header_row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(main_headers)
            data_rows = defense_table.find('tbody').find_all('tr')
            for row in data_rows:
                data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(data)
    except Exception as e:
        print(f"An error occurred while writing the defense fumbles data CSV file: {e}")

    # Touchdown Log
    try:
        td_log_div = soup.find('div', id='all_team_td_log')
        td_log_table = BeautifulSoup(str(td_log_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
        with open(f'./data-teams/{team_name}_{year}_touchdown_log.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                header_row = td_log_table.find('tr')
                headers = [col.text for col in header_row.find_all('th') if col.text.strip()]
                writer.writerow(headers)
            data_rows = td_log_table.find('tbody').find_all('tr')
            for row in data_rows:
                data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(data)
    except Exception as e:
        print(f"An error occurred while writing the touchdown log CSV file: {e}")

    # Opponent Touchdown Log
    try:
        opp_td_log_div = soup.find('div', id='all_opp_td_log')
        opp_td_log_table = BeautifulSoup(str(opp_td_log_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
        with open(f'./data-teams/{team_name}_{year}_opp_td_log_data.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                main_header_row = opp_td_log_table.find('tr')
                main_headers = [col.text for col in main_header_row.find_all('th') if col.text.strip()]
                writer.writerow(main_headers)
            data_rows = opp_td_log_table.find('tbody').find_all('tr')
            for row in data_rows:
                data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(data)
    except Exception as e:
        print(f"An error occurred while writing the opponent touchdown log CSV file: {e}")

    # Kick & Punt Returns
    try:
        returns_div = soup.find('div', id='all_returns')
        returns_table = BeautifulSoup(str(returns_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
        with open(f'./data-teams/{team_name}_{year}_kick_punt_returns_data.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                main_header_rows = returns_table.find_all('tr', class_='over_header')
                main_headers = [col.text for row in main_header_rows for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(main_headers)
                sub_header_row = returns_table.find_all('tr')[len(main_header_rows)]
                sub_headers = [col.text for col in sub_header_row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(sub_headers)
            data_rows = returns_table.find('tbody').find_all('tr')
            for row in data_rows:
                data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(data)
    except Exception as e:
        print(f"An error occurred while writing the kick punt returns data CSV file: {e}")

    # Scoring Summary
    try:
        scoring_div = soup.find('div', id='all_scoring')
        scoring_table = BeautifulSoup(str(scoring_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
        with open(f'./data-teams/{team_name}_{year}_scoring_summary_data.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                main_header_row = scoring_table.find('tr')
                main_headers = [col.text for col in main_header_row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(main_headers)
            data_rows = scoring_table.find('tbody').find_all('tr')
            for row in data_rows:
                data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
                writer.writerow(data)
    except Exception as e:
        print(f"An error occurred while writing the scoring summary CSV file: {e}")


In [5]:
nfl_teams = [
    "Arizona Cardinals",
    "Atlanta Falcons",
    "Baltimore Ravens",
    "Buffalo Bills",
    "Carolina Panthers",
    "Chicago Bears",
    "Cincinnati Bengals",
    "Cleveland Browns",
    "Dallas Cowboys",
    "Denver Broncos",
    "Detroit Lions",
    "Green Bay Packers",
    "Houston Texans",
    "Indianapolis Colts",
    "Jacksonville Jaguars",
    "Kansas City Chiefs",
    "Las Vegas Raiders",
    "Los Angeles Chargers",
    "Los Angeles Rams",
    "Miami Dolphins",
    "Minnesota Vikings",
    "New England Patriots",
    "New Orleans Saints",
    "New York Giants",
    "New York Jets",
    "Philadelphia Eagles",
    "Pittsburgh Steelers",
    "San Francisco 49ers",
    "Seattle Seahawks",
    "Tampa Bay Buccaneers",
    "Tennessee Titans",
    "Washington Football Team"
]

#headers = ['Week', 'Day', 'Date', 'OT', 'Rec', 'Opp', 'Tm', 'Opp', '1stD', 'TotYd', 'PassY', 'RushY', 'TO',
#           '1stD', 'TotYd', 'PassY', 'RushY', 'TO', 'Offense', 'Defense', 'Sp. Tms']

base_url = "https://www.pro-football-reference.com/teams/"
team_codes = ["crd", "atl", "rav", "buf", "car", "chi", "cin", "cle", "dal", "den", "det", "gnb", "htx",
              "clt", "jax", "kan", "rai", "sdg", "ram", "mia", "min", "nwe", "nor", "nyg", "nyj", "phi",
              "pit", "sfo", "sea", "tam", "oti", "was"]

# Loop through each year
for year in range(2005, 2015):
    # Loop through each team
    for team, team_code in tqdm(zip(nfl_teams, team_codes), desc=f"Year {year}", leave=False):
        url = f"{base_url}{team_code}/{year}.htm"
        scrape_data(url, year, team)
        time.sleep(5)

print("Scraping complete.")

Year 2005: 0it [00:00, ?it/s]

Year 2006: 0it [00:00, ?it/s]

Year 2007: 0it [00:00, ?it/s]

Year 2008: 0it [00:00, ?it/s]

Year 2009: 0it [00:00, ?it/s]

...

Year 2011: 0it [00:00, ?it/s]

Year 2012: 0it [00:00, ?it/s]

Year 2013: 0it [00:00, ?it/s]

Year 2014: 0it [00:00, ?it/s]

Scraping complete.


# LOOP

In [ ]:
def scrape_data(url, year, team_name):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Team General
    details = {
        "Record": soup.find('p').text,
        "Coach": soup.find_all('p')[1].text,
        "Points For": soup.find_all('p')[2].text,
        "Points Against": soup.find_all('p')[3].text,
        "Expected W-L": soup.find_all('p')[4].text,
        "SRS/SOS": soup.find_all('p')[5].text,
        "Playoffs": soup.find_all('p')[6].text,
        "Offensive Coordinator": soup.find_all('p')[7].text,
        "Defensive Coordinator": soup.find_all('p')[8].text,
        "Other Notable Asst.": soup.find_all('p')[9].text,
        "Stadium": soup.find_all('p')[10].text,
        "Owner/President/GM": soup.find_all('p')[11].text,
        "Offensive Scheme": soup.find_all('p')[12].text,
        "Defensive Alignment": soup.find_all('p')[13].text,
        "Preseason Odds": soup.find_all('p')[14].text,
        "Training Camp": soup.find_all('p')[15].text
    }
    with open(f'./data-teams/{team_name}_{year}_team_general.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            writer.writerow(details.keys())
        writer.writerow(details.values())

    # Team Stats and Rankings
    table = soup.find('table', {'id': 'team_stats'})
    headers = [th.text for th in table.find('thead').find_all('tr')[1].find_all('th')]
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        row = [td.text for td in tr.find_all('td')]
        rows.append(row)
    with open(f'./data-teams/{team_name}_{year}_team_stats.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            writer.writerow(headers[1:])
        writer.writerows(rows)

    # Teams Schedule and Game Results
    table = soup.find_all('table')[1]
    rows = table.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text for col in cols]
        data.append(cols)
    with open(f'./data-teams/{team_name}_{year}_team_schedule.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            writer.writerow(headers)
        writer.writerows(data)

    # Team Conversions
    table = soup.find('table', {'id': 'team_conversions'})
    headers = [th.text for th in table.find('thead').find_all('th')]
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        row = [td.text for td in tr.find_all('td')]
        rows.append(row)
    with open(f'./data-teams/{team_name}_{year}_team_conversions.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            writer.writerow(headers[1:])
        writer.writerows(rows)

    # Passing Stats
    passing_div = soup.find('div', id='all_passing')
    passing_table = BeautifulSoup(str(passing_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
    with open(f'./data-teams/{team_name}_{year}_passing_data.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            main_header_row = passing_table.find('tr')
            main_headers = [col.text for col in main_header_row.find_all('th') if col.text.strip()]
            writer.writerow(main_headers)
        data_rows = passing_table.find('tbody').find_all('tr')
        for row in data_rows:
            data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(data)

    # Other sections scraping and saving logic can be added here...

    # ... (previous code)

    # Rushing and Receiving
    rushing_receiving_div = soup.find('div', id='all_rushing_and_receiving')
    rushing_receiving_table = BeautifulSoup(str(rushing_receiving_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
    with open(f'./data-teams/{team_name}_{year}_rushing_receiving_data.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            main_header_row = rushing_receiving_table.find_all('tr')[1] # The second row contains the main headers
            main_headers = [col.text for col in main_header_row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(main_headers)
        data_rows = rushing_receiving_table.find('tbody').find_all('tr')
        for row in data_rows:
            data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(data)

    # Kicking
    kicking_div = soup.find('div', id='all_kicking')
    kicking_table = BeautifulSoup(str(kicking_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
    with open(f'./data-teams/{team_name}_{year}_kicking_data.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            main_header_row = kicking_table.find('tr')
            main_headers = [col.text for col in main_header_row.find_all('th') if col.text.strip()]
            writer.writerow(main_headers)
        data_rows = kicking_table.find('tbody').find_all('tr')
        for row in data_rows:
            data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(data)

    # Punting
    punting_div = soup.find('div', id='all_punting')
    punting_table = BeautifulSoup(str(punting_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
    with open(f'./data-teams/{team_name}_{year}_punting_data.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            main_header_row = punting_table.find('tr')
            main_headers = [col.text for col in main_header_row.find_all('th') if col.text.strip()]
            writer.writerow(main_headers)
        data_rows = punting_table.find('tbody').find_all('tr')
        for row in data_rows:
            data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(data)

    # Defense and Fumbles
    defense_div = soup.find('div', id='all_defense')
    defense_table = BeautifulSoup(str(defense_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
    with open(f'./data-teams/{team_name}_{year}_defense_fumbles_data.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            main_header_row = defense_table.find_all('tr')[2] # The third row contains the main headers
            main_headers = [col.text for col in main_header_row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(main_headers)
        data_rows = defense_table.find('tbody').find_all('tr')
        for row in data_rows:
            data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(data)

    # Touchdown Log
    td_log_div = soup.find('div', id='all_team_td_log')
    td_log_table = BeautifulSoup(str(td_log_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
    with open(f'./data-teams/{team_name}_{year}_touchdown_log.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            header_row = td_log_table.find('tr')
            headers = [col.text for col in header_row.find_all('th') if col.text.strip()]
            writer.writerow(headers)
        data_rows = td_log_table.find('tbody').find_all('tr')
        for row in data_rows:
            data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(data)

    # Opponent Touchdown Log
    opp_td_log_div = soup.find('div', id='all_opp_td_log')
    opp_td_log_table = BeautifulSoup(str(opp_td_log_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
    with open(f'./data-teams/{team_name}_{year}_opp_td_log_data.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            main_header_row = opp_td_log_table.find('tr')
            main_headers = [col.text for col in main_header_row.find_all('th') if col.text.strip()]
            writer.writerow(main_headers)
        data_rows = opp_td_log_table.find('tbody').find_all('tr')
        for row in data_rows:
            data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(data)

    # Kick & Punt Returns
    returns_div = soup.find('div', id='all_returns')
    returns_table = BeautifulSoup(str(returns_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
    with open(f'./data-teams/{team_name}_{year}_kick_punt_returns_data.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            main_header_rows = returns_table.find_all('tr', class_='over_header')
            main_headers = [col.text for row in main_header_rows for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(main_headers)
            sub_header_row = returns_table.find_all('tr')[len(main_header_rows)]
            sub_headers = [col.text for col in sub_header_row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(sub_headers)
        data_rows = returns_table.find('tbody').find_all('tr')
        for row in data_rows:
            data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(data)

    # Scoring Summary
    scoring_div = soup.find('div', id='all_scoring')
    scoring_table = BeautifulSoup(str(scoring_div).split('<!--')[1].split('-->')[0], 'html.parser').find('table')
    with open(f'./data-teams/{team_name}_{year}_scoring_summary_data.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if csvfile.tell() == 0:
            main_header_row = scoring_table.find('tr')
            main_headers = [col.text for col in main_header_row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(main_headers)
        data_rows = scoring_table.find('tbody').find_all('tr')
        for row in data_rows:
            data = [col.text for col in row.find_all(['th', 'td']) if col.text.strip()]
            writer.writerow(data)


In [ ]:
nfl_teams = [
    "Arizona Cardinals",
    "Atlanta Falcons",
    "Baltimore Ravens",
    "Buffalo Bills",
    "Carolina Panthers",
    "Chicago Bears",
    "Cincinnati Bengals",
    "Cleveland Browns",
    "Dallas Cowboys",
    "Denver Broncos",
    "Detroit Lions",
    "Green Bay Packers",
    "Houston Texans",
    "Indianapolis Colts",
    "Jacksonville Jaguars",
    "Kansas City Chiefs",
    "Las Vegas Raiders",
    "Los Angeles Chargers",
    "Los Angeles Rams",
    "Miami Dolphins",
    "Minnesota Vikings",
    "New England Patriots",
    "New Orleans Saints",
    "New York Giants",
    "New York Jets",
    "Philadelphia Eagles",
    "Pittsburgh Steelers",
    "San Francisco 49ers",
    "Seattle Seahawks",
    "Tampa Bay Buccaneers",
    "Tennessee Titans",
    "Washington Football Team"
]

headers = ['Week', 'Day', 'Date', 'OT', 'Rec', 'Opp', 'Tm', 'Opp', '1stD', 'TotYd', 'PassY', 'RushY', 'TO',
           '1stD', 'TotYd', 'PassY', 'RushY', 'TO', 'Offense', 'Defense', 'Sp. Tms']

base_url = "https://www.pro-football-reference.com/teams/"
team_codes = ["crd", "atl", "rav", "buf", "car", "chi", "cin", "cle", "dal", "den", "det", "gnb", "htx",
              "clt", "jax", "kan", "rai", "sdg", "ram", "mia", "min", "nwe", "nor", "nyg", "nyj", "phi",
              "pit", "sfo", "sea", "tam", "oti", "was"]

for year in range(2022, 2023):
    # Loop through each team
    for team, team_code in tqdm(zip(nfl_teams, team_codes), desc=f"Year {year}", leave=False):
        url = f"{base_url}{team_code}/{year}.htm"

        try:
            scrape_data(url, year, team)
        except AttributeError:
            file_name = f"./data-teams/{team}_{year}_team_general.csv"
            print(f"Error: Table not found for {file_name}")

print("Scraping complete.")